In [23]:
from backtest_lib.market.polars_impl import SeriesUniverseMapping

vec1 = SeriesUniverseMapping.from_vectors(["a", "b", "c"], [1, 2, 3])
vec2 = SeriesUniverseMapping.from_vectors(["a", "b", "c"], [1, 2, 3])
vec1 + vec2

SeriesUniverseMapping(names=('a', 'b', 'c'), _data=shape: (3,)
Series: '' [i64]
[
	2
	4
	6
], _scalar_type=<class 'int'>)

In [24]:
vec3 = SeriesUniverseMapping.from_vectors(["a", "c", "b"], [0, 0, 1])
res = vec1 + vec3
print(res)
assert dict(res) == {"a": 1, "b": 3, "c": 3}

SeriesUniverseMapping(names=('a', 'b', 'c'), _data=shape: (3,)
Series: '' [i64]
[
	1
	3
	3
], _scalar_type=<class 'int'>)


In [25]:
try:
    vec4 = SeriesUniverseMapping.from_vectors(["a", "c", "d"], [0, 0, 1])
    vec1 + vec4
    assert False
except:
    print("didn't work. that is expected, keys are different.")

didn't work. that is expected, keys are different.


In [26]:
vec5 = SeriesUniverseMapping.from_vectors(["a", "b"], [0, 1])
res = vec1 + vec5
print(res)
assert dict(res) == {"a": 1, "b": 3, "c": 3}


SeriesUniverseMapping(names=('a', 'b', 'c'), _data=shape: (3,)
Series: '' [i64]
[
	1
	3
	3
], _scalar_type=<class 'int'>)


In [27]:
vec6 = SeriesUniverseMapping.from_vectors(["a", "b"], [2, 2])
res = vec1 * vec6
print(res)

# NOTE: Interesting case here, should "c" be 0 or 3?
assert dict(res) == {"a": 2, "b": 4, "c": 0}

SeriesUniverseMapping(names=('a', 'b', 'c'), _data=shape: (3,)
Series: '' [i64]
[
	2
	4
	0
], _scalar_type=<class 'int'>)


In [32]:
vec7 = SeriesUniverseMapping.from_vectors(["a", "c"], [2, 2])
res = vec1 / vec7
print(res)

# NOTE: Another sharp edge, should we implicitly pad RHS with 0s?
# It feels weirdly un-ergonomic for division.
assert dict(res) == {"a": 0.5, "b": float("inf"), "c": 1.5}

SeriesUniverseMapping(names=('a', 'b', 'c'), _data=shape: (3,)
Series: '' [f64]
[
	0.5
	inf
	1.5
], _scalar_type=<class 'float'>)
